Chargement des données et préparation des variables.

In [ ]:
import pandas as pd
import json

INPUT_FOLDER = '../1_data/1_raw/'

OUTPUT_FOLDER = '../1_data/2_preprocessed/'

dataset = pd.read_csv(INPUT_FOLDER + 'adult_census_income.csv')

# On ne souhaite pas utiliser la colonne 'race'
dataset = dataset.drop(columns='race')

# la colonne fnlwgt (final weight) représente la pondération des individus dans la population.
# Cette variable ne doit jamais être utilisée pour l'entrainement d'un modèle, elle ne décrit pas l'individu. 
# Cette variable devrait être prise en compte comme poids dans l'entrainement d'un modèle.
# Ici, on ne souhaite que consider les variables sur lesquelles le modèle apprend. On peut donc la retirer.
dataset = dataset.drop(columns='fnlwgt')

# Il existe deux variables pour qualifier le niveau d'études : education et education.num.
# Elles sont équivalentes mais la première n'est pas numérique, on conserve donc la seconde.
dataset = dataset.drop(columns='education')

# Les variables capital.gain et capital.loss indiquent tous les deux une variation du capital (investissement) mais en positif et en négatif. 
# On peut simplifier ces deux features en une seule.
dataset['capital_diff'] = dataset['capital.gain'] - dataset['capital.loss']
dataset = dataset.drop(columns=['capital.gain', 'capital.loss'])

# Les valeurs manquantes sont remplacées par des "Unknown"
dataset.replace('?', pd.NA, inplace=True)
for col in dataset.columns:
    dataset[col] = dataset[col].fillna('Unknown')

# Transformation des variables qualitatives en variables numériques (one-hot encoding).
# workclass, marital.status, occupation, relationship, sex, native.country, income.
# sex et income sont binaires, les autres multiclasse.
dataset['sex'] = dataset.sex == 'Male'
dataset['income'] = dataset.income == '>50K'
dataset = pd.get_dummies(dataset, columns=['workclass', 'marital.status', 'occupation', 'relationship', 'native.country'])

# On transforme les colonnes de type int en type float pour éviter les erreurs en cas de valeur manquante à l'avenir
int_cols = dataset.select_dtypes(include="int").columns
dataset[int_cols] = dataset[int_cols].astype("float64")

# On enregistre le jeu de donnée préparé.
dataset.to_csv(OUTPUT_FOLDER + 'x0.csv')
x0_param = {
    'sample_name' : 'x0',
    'sample_strategy': None,
}
with open(OUTPUT_FOLDER + "metadata.json", "w", encoding="utf-8") as f:
    json.dump(x0_param, f, indent=4, ensure_ascii=False)

On souhaite ensuite créer des échantillons biaisés.

Pour les construire, on va classer les individus dans des strates selon "sex" et "income".

In [ ]:
dataset['strate'] = dataset['sex'].astype(str) + '_' + dataset['income'].astype(str)
dataset['strate'].value_counts()

strate
True_False     15128
False_False     9592
True_True       6662
False_True      1179
Name: count, dtype: int64

Création de huit échantillons :

Le premier est une simple réduction du jeu de données, les autres sont construit pour créer des situations de fairness à partir des trois critères suivant :
* (a) : Equilibrer la variable "sex"
* (b) : Equilibrer la variable "income"
* (c) : Equilibrer les proportions "sex"/"income" (Homme>50k / Homme<=50k = Femme>50k / Femme<=50k)

Afin de pouvoir comparer des modèles de machine learning entrainés sur ces échantillons, on pose une taille de 4000 pour chacun d'entre eux.

Ci-dessous ces huits échantillons présentés sous la forme d'un tableau d'effectifs :

X1

![X1](img/x1.jpg)

A noter, X1 n'est pas vraiment représentatif du dataset adult income car nous avons retiré les pondérations fnlwgt.

X2

![X2](img/x2.jpg)

X3

![X3](img/x3.jpg)

X4

![X4](img/x4.jpg)

X5

![X5](img/x5.jpg)

X6

![X6](img/x6.jpg)

X7

![X7](img/x7.jpg)

X8

![X8](img/x8.jpg)

In [ ]:
def sample(data, minf, finf, msup, fsup):
    strates = {
        'True_False': minf,
        'False_False': finf,
        'True_True': msup,
        'False_True': fsup
    }
    X = []
    for strate, size in strates.items():
        group = data[data['strate'] == strate]
        if len(group) < size:
            raise ValueError(f"Pas assez d'observations dans la strate {strate} (dispo: {len(group)}, demandé: {size})")
        X.append(group.sample(n=size, random_state=1))
    X = pd.concat(X).reset_index(drop=True)
    X = X.drop(columns='strate')
    return X

In [ ]:
# x1_param = {
#     'sample_name' : 'x1',
#     'sample_strategy': {
#         'male_<=50k' : 1859,
#         'female_<=50k' : 1178,
#         'male_>50k' : 818,
#         'female_>50k' : 145,
#     }
# }
# X1 = sample(dataset, 1859, 1178, 818, 145)
# x2_param = {
#     'sample_name' : 'x2',
#     'sample_strategy': {
#         'male_<=50k' : 1400,
#         'female_<=50k' : 1600,
#         'male_>50k' : 600,
#         'female_>50k' : 400,
#     }
# }
# X2 = sample(dataset, 1400, 1600, 600, 400)
# x3_param = {
#     'sample_name' : 'x3',
#     'sample_strategy': {
#         'male_<=50k' : 1300,
#         'female_<=50k' : 700,
#         'male_>50k' : 1700,
#         'female_>50k' : 300,
#     }
# }
# X3 = sample(dataset, 1300, 700, 1700, 300)
# x4_param = {
#     'sample_name' : 'x4',
#     'sample_strategy': {
#         'male_<=50k' : 2031,
#         'female_<=50k' : 1005,
#         'male_>50k' : 645,
#         'female_>50k' : 319,
#     }
# }
# X4 = sample(dataset, 2031, 1005, 645, 319)
# x5_param = {
#     'sample_name' : 'x5',
#     'sample_strategy': {
#         'male_<=50k' : 800,
#         'female_<=50k' : 1200,
#         'male_>50k' : 1200,
#         'female_>50k' : 800,
#     }
# }
# X5 = sample(dataset, 800, 1200, 1200, 800)
# x6_param = {
#     'sample_name' : 'x6',
#     'sample_strategy': {
#         'male_<=50k' : 1500,
#         'female_<=50k' : 1500,
#         'male_>50k' : 500,
#         'female_>50k' : 500,
#     }
# }
# X6 = sample(dataset, 1500, 1500, 500, 500)
# x7_param = {
#     'sample_name' : 'x7',
#     'sample_strategy': {
#         'male_<=50k' : 1300,
#         'female_<=50k' : 700,
#         'male_>50k' : 1300,
#         'female_>50k' : 700,
#     }
# }
# X7 = sample(dataset, 1300, 700, 1300, 700)
# x8_param = {
#     'sample_name' : 'x8',
#     'sample_strategy': {
#         'male_<=50k' : 1000,
#         'female_<=50k' : 1000,
#         'male_>50k' : 1000,
#         'female_>50k' : 1000,
#     }
# }
# X8 = sample(dataset, 1000, 1000, 1000, 1000)

In [ ]:
# X1.to_csv(OUTPUT_FOLDER + 'x1.csv')
# with open(OUTPUT_FOLDER + "metadata.json", "w", encoding="utf-8") as f:
#     json.dump(x1_param, f, indent=4, ensure_ascii=False)
# X2.to_csv(OUTPUT_FOLDER + 'x2.csv')
# with open(OUTPUT_FOLDER + "metadata.json", "w", encoding="utf-8") as f:
#     json.dump(x2_param, f, indent=4, ensure_ascii=False)
# X3.to_csv(OUTPUT_FOLDER + 'x3.csv')
# with open(OUTPUT_FOLDER + "metadata.json", "w", encoding="utf-8") as f:
#     json.dump(x3_param, f, indent=4, ensure_ascii=False)
# X4.to_csv(OUTPUT_FOLDER + 'x4.csv')
# with open(OUTPUT_FOLDER + "metadata.json", "w", encoding="utf-8") as f:
#     json.dump(x4_param, f, indent=4, ensure_ascii=False)
# X5.to_csv(OUTPUT_FOLDER + 'x5.csv')
# with open(OUTPUT_FOLDER + "metadata.json", "w", encoding="utf-8") as f:
#     json.dump(x5_param, f, indent=4, ensure_ascii=False)
# X6.to_csv(OUTPUT_FOLDER + 'x6.csv')
# with open(OUTPUT_FOLDER + "metadata.json", "w", encoding="utf-8") as f:
#     json.dump(x6_param, f, indent=4, ensure_ascii=False)
# X7.to_csv(OUTPUT_FOLDER + 'x7.csv')
# with open(OUTPUT_FOLDER + "metadata.json", "w", encoding="utf-8") as f:
#     json.dump(x7_param, f, indent=4, ensure_ascii=False)
# X8.to_csv(OUTPUT_FOLDER + 'x8.csv')
# with open(OUTPUT_FOLDER + "metadata.json", "w", encoding="utf-8") as f:
#     json.dump(x8_param, f, indent=4, ensure_ascii=False)